In [6]:
import numpy as np
import pickle

class MLP:
    def __init__(self, learning_rate=0.1, epochs=100, hidden_nodes=2):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.hidden_weights = None
        self.hidden_bias = None
        self.output_weights = None
        self.output_bias = None
        self.hidden_nodes = hidden_nodes

    def save_model(self, filename):
        model_data = {
            'learning_rate': self.learning_rate,
            'epochs': self.epochs,
            'hidden_nodes': self.hidden_nodes,
            'hidden_weights': self.hidden_weights.tolist(),
            'hidden_bias': self.hidden_bias.tolist(),
            'output_weights': self.output_weights.tolist(),
            'output_bias': self.output_bias.tolist()
        }
        with open(filename, 'wb') as f:
            pickle.dump(model_data, f)

    @staticmethod
    def load_model(filename):
        with open(filename, 'rb') as f:
            model_data = pickle.load(f)
        mlp = MLP(
            learning_rate=model_data['learning_rate'],
            epochs=model_data['epochs'],
            hidden_nodes=model_data['hidden_nodes']
        )
        mlp.hidden_weights = np.array(model_data['hidden_weights'])
        mlp.hidden_bias = np.array(model_data['hidden_bias'])
        mlp.output_weights = np.array(model_data['output_weights'])
        mlp.output_bias = np.array(model_data['output_bias'])
        return mlp

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        hidden_output = self.sigmoid(np.dot(inputs, self.hidden_weights) + self.hidden_bias)
        output = self.sigmoid(np.dot(hidden_output, self.output_weights) + self.output_bias)
        return output

    def train(self, training_inputs, labels):
        self.hidden_weights = np.random.rand(training_inputs.shape[1], self.hidden_nodes)
        self.hidden_bias = np.random.rand(self.hidden_nodes)
        self.output_weights = np.random.rand(self.hidden_nodes, 1)
        self.output_bias = np.random.rand(1)

        for _ in range(self.epochs):
            for inputs, label in zip(training_inputs, labels):

                hidden_output = self.sigmoid(np.dot(inputs, self.hidden_weights) + self.hidden_bias)
                output = self.sigmoid(np.dot(hidden_output, self.output_weights) + self.output_bias)

                output_error = label - output
                output_delta = output_error * output * (1 - output)

                hidden_error = np.dot(output_delta, self.output_weights.T)
                hidden_delta = hidden_error * hidden_output * (1 - hidden_output)

                self.output_weights += self.learning_rate * np.dot(hidden_output.reshape(-1, 1), output_delta.reshape(1, -1))
                self.output_bias += self.learning_rate * output_delta

                self.hidden_weights += self.learning_rate * np.dot(inputs.reshape(-1, 1), hidden_delta.reshape(1, -1))
                self.hidden_bias += self.learning_rate * hidden_delta

training_inputs = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
labels = np.array([[0], [1], [1], [0]])

mlp = MLP(learning_rate=0.5, epochs=5000)
mlp.train(training_inputs, labels)

print("Результат:")
for inputs in training_inputs:
    print(f"{inputs} -> {mlp.predict(inputs).round()}")
mlp.save_model("weights")

Результат:
[0 0] -> [0.]
[0 1] -> [1.]
[1 0] -> [1.]
[1 1] -> [0.]
